## Combined models
1) rf-benign-specialize
2) rf-general
3) rf-lexical

In [349]:
import os
import pickle

from collections import defaultdict
import numpy as np
import pandas as pd 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split


In [350]:
def classification_type(type):
    '''
    Convert classification type into values:
        1) Benign = 0
        2) Defacement = 1
        3) Phishing = 2
        4) Malware = 3    
    '''
    if type == "benign":
        return 0
    elif type == "defacement":
        return 1
    elif type == "phishing":
        return 2
    elif type == "malware":
        return 3
    else:
        print(f"Unable to find proper type: {type}")


def calc_FNR_accuracy(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    for label_class in range(4):
        FN = sum(conf_matrix[label_class][i] for i in range(len(conf_matrix)) if i != label_class)  
        
        TP = conf_matrix[label_class][label_class]  
        
        TN = np.sum(np.delete(np.delete(conf_matrix, label_class, axis=0), label_class, axis=1))
        
        accuracy = (TP + TN) / np.sum(conf_matrix)
        print("Accuracy for class", label_class, ":", accuracy)

        FNR = FN / (FN + TP) if (FN + TP) > 0 else -1
        print("FNR for class", label_class, ":", FNR)

In [351]:
# Load models
model_files = [file for file in os.listdir(os.getcwd()) if file.endswith(".pkl")]
models = []
for file in model_files:
    with open(file, "rb") as file:
        models.append((file.name, pickle.load(file)))

In [352]:
# Data for training ensemble model
training_dataset = {'rf-benign-specialize.pkl': 'rf-benign-specialize-features.csv',
                    'rf-general.pkl': 'rf-general-features.csv',
                    'rf-lexical.pkl': 'rf-lexical-features.csv'}

df = pd.read_csv('../datasets/malicious_phish.csv')
y = df.iloc[:, 1]
y = y.apply(classification_type)

In [353]:
testing_ensemble_X = {} # test size 0.2 values
testing_ensemble_y = None # test size 0.2 values

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    data = pd.read_csv(training_dataset[model_name], header=None, skiprows=1)
    _, X_test, _, y_test = train_test_split(data, y, test_size=0.2, random_state=69)

    # y_pred = model.predict(X_train) # prediction for model
    # if input_ensemble is None:
    #     input_ensemble = np.empty((y_pred.shape[0], 3))

    if testing_ensemble_y is None:
        testing_ensemble_y = y_test

    # input_ensemble[:, i] = y_pred
    testing_ensemble_X[model_name] = X_test

In [354]:
# Attached to endpoint prediction of above models
# priority = [('rf-general.pkl', 3), ('rf-lexical.pkl', 3), ('rf-lexical.pkl', 1),
#             ('rf-benign-specialize.pkl', 3), ('rf-benign-specialize.pkl', 0), ('rf-general.pkl', 1),
#             ('rf-general.pkl', 2), ('rf-benign-specialize.pkl', 1), ('rf-lexical.pkl', 0),
#             ('rf-lexical.pkl', 2), ('rf-benign-specialize.pkl', 2), ('rf-general.pkl', 0)]

priority = [('rf-general.pkl', 3), ('rf-benign-specialize.pkl', 0), ('rf-general.pkl', 2), ('rf-general.pkl', 1), ('rf-lexical.pkl', 1), ('rf-lexical.pkl', 3),
            ('rf-benign-specialize.pkl', 3),
            ('rf-benign-specialize.pkl', 1), ('rf-lexical.pkl', 0),
            ('rf-lexical.pkl', 2), ('rf-benign-specialize.pkl', 2), ('rf-general.pkl', 0)]

### Test on 0.2 validation split initial dataset

In [355]:
y_ensemble_pred = None
model_col = {} # Takes note of which column the model predictions are in

for i in range(len(models)):
    model_name, model = models[i]
    X_test = testing_ensemble_X[model_name]

    y_pred = model.predict(X_test)
    if y_ensemble_pred is None:
        y_ensemble_pred = np.empty((y_pred.shape[0], 3))

    y_ensemble_pred[:, i] = y_pred
    model_col[i] = model_name

# Look at priority list
pred_combined_y = np.empty(shape=testing_ensemble_y.shape)
row = 0
for entry in y_ensemble_pred:
    # Match to model
    statements = defaultdict(int)

    for i, prediction in enumerate(entry):
        statements[(model_col[i], prediction)] = 1
    for p in priority:
        if statements[p] == 1:
            # Take the choice
            choice_for_entry = p[1]
            pred_combined_y[row] = choice_for_entry
            break
    row += 1


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [356]:
precision = precision_score(testing_ensemble_y, pred_combined_y, average='weighted')
recall = recall_score(testing_ensemble_y, pred_combined_y, average='weighted')
f1 = f1_score(testing_ensemble_y, pred_combined_y, average='weighted')
val_accuracy = accuracy_score(testing_ensemble_y, pred_combined_y)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(testing_ensemble_y, pred_combined_y)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(testing_ensemble_y, pred_combined_y)

OVERALL: Accuracy: 0.67873678, Precision: 0.62674700, Recall: 0.67873678, F1 Score: 0.56710771
Class 0:	Test Precision: 0.67248733,	Test Recall: 0.99954466,	Test f1: 0.80402902
Class 1:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 2:	Test Precision: 0.93137255,	Test Recall: 0.10657621,	Test f1: 0.19126600
Class 3:	Test Precision: 1.00000000,	Test Recall: 0.12010919,	Test f1: 0.21445979
Accuracy for class 0 : 0.679566028608942
FNR for class 0 : 0.0004553415061295972
Accuracy for class 1 : 0.851995178095655
FNR for class 1 : 1.0
Accuracy for class 2 : 0.8704612289713527
FNR for class 2 : 0.8934237940060901
Accuracy for class 3 : 0.9554511321493562
FNR for class 3 : 0.8798908098271155


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Test on phishing dataset

In [357]:
training_dataset_phishing = {'rf-benign-specialize.pkl': 'rf-benign-specialize-test-phishing.csv',
                    'rf-general.pkl': 'rf-general-test-phishing.csv',
                    'rf-lexical.pkl': 'rf-lexical-test-phishing.csv'}

input_ensemble_phishing = None

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    X_test = pd.read_csv(training_dataset_phishing[model_name], header=None, skiprows=1)

    y_pred = model.predict(X_test) # prediction for model
    if input_ensemble_phishing is None:
        input_ensemble_phishing = np.empty((y_pred.shape[0], 3))

    input_ensemble_phishing[:, i] = y_pred

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [358]:
# Phishing is label 2.
y_phishing = np.full((input_ensemble_phishing.shape[0],), 2)
pred_combined_y_phishing = np.empty(shape=y_phishing.shape)
row = 0
for entry in input_ensemble_phishing:
    # Match to model
    statements = defaultdict(int)

    for i, prediction in enumerate(entry):
        statements[(model_col[i], prediction)] = 1
    for p in priority:
        if statements[p] == 1:
            # Take the choice
            choice_for_entry = p[1]
            pred_combined_y_phishing[row] = choice_for_entry
            break
    row += 1

In [359]:
precision = precision_score(y_phishing, pred_combined_y_phishing, average='weighted')
recall = recall_score(y_phishing, pred_combined_y_phishing, average='weighted')
f1 = f1_score(y_phishing, pred_combined_y_phishing, average='weighted')
val_accuracy = accuracy_score(y_phishing, pred_combined_y_phishing)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(y_phishing, pred_combined_y_phishing)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(y_phishing, pred_combined_y_phishing)

OVERALL: Accuracy: 0.00000000, Precision: 0.00000000, Recall: 0.00000000, F1 Score: 0.00000000
Class 0:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 1:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 2:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize(

IndexError: index 3 is out of bounds for axis 0 with size 3

### Test on benign dataset

In [ ]:
training_dataset_benign = {'rf-benign-specialize.pkl': 'rf-benign-specialize-test-benign.csv',
                    'rf-general.pkl': 'rf-general-test-benign.csv',
                    'rf-lexical.pkl': 'rf-lexical-test-benign.csv'}

input_ensemble_benign = None

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    X_test = pd.read_csv(training_dataset_benign[model_name], header=None, skiprows=1)

    y_pred = model.predict(X_test) # prediction for model
    if input_ensemble_benign is None:
        input_ensemble_benign = np.empty((y_pred.shape[0], 3))

    input_ensemble_benign[:, i] = y_pred

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# Benign is label 0.
y_benign = np.full((input_ensemble_benign.shape[0],), 2)
pred_combined_y_benign = np.empty(shape=y_benign.shape)
row = 0
for entry in input_ensemble_benign:
    # Match to model
    statements = defaultdict(int)

    for i, prediction in enumerate(entry):
        statements[(model_col[i], prediction)] = 1
    for p in priority:
        if statements[p] == 1:
            # Take the choice
            choice_for_entry = p[1]
            pred_combined_y_benign[row] = choice_for_entry
            break
    row += 1

In [ ]:
precision = precision_score(y_benign, pred_combined_y_benign, average='weighted')
recall = recall_score(y_benign, pred_combined_y_benign, average='weighted')
f1 = f1_score(y_benign, pred_combined_y_benign, average='weighted')
val_accuracy = accuracy_score(y_benign, pred_combined_y_benign)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(y_benign, pred_combined_y_benign)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(y_benign, pred_combined_y_benign)

OVERALL: Accuracy: 0.00000000, Precision: 0.00000000, Recall: 0.00000000, F1 Score: 0.00000000
Class 0:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 1:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 2:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize(

IndexError: index 3 is out of bounds for axis 0 with size 3